In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.signal import find_peaks, spectrogram
from scipy.signal import butter, lfilter
from scipy.signal import hilbert
from tensorflow import keras
import math

%matplotlib qt

In [3]:
mpl.rcParams['agg.path.chunksize'] = 100_000

LOWCUT = 34_500
HIGHCUT = 35_500
PING_FREQ = 35_000
BANDPASS_WIDTHS = [4_000, 2_000, 1000, 750, 500]
BANDPASS_WIDTH = 1000
LOWPASS_CUTOFF = 6_000 # kinda arbitrarily chosen. In practice, cutoff for envelope detection should be square root of carrier freq (35 kHz) and message freq (not quite sure, chose 1 kHz?) TODO: Run fft on data and see what the frequency spectrum looks like
SOUND_SPEED = 1500

SAMPLE_RATE = 625_000

2024-01-26 14:33:01.854 Python[85485:7292811] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/bw/4tzf4sn540g21cjq2sgsty4w0000gn/T/org.python.python.savedState


## Filter Functions

In [4]:
def butter_bandpass(lowcut, highcut, fs=SAMPLE_RATE, order=5):
    """
    Generates the coefficients for a Butterworth bandpass filter.

    Parameters
    ----------
    lowcut : int
        The lower frequency cutoff

    highcut : int
        The higher frequency cutoff

    fs : int
        The sample rate

    order : int
        The order of the filter

    Returns
    -------
    b : np.array
        The numerator coefficients

    a : np.array
        The denominator coefficients
    """

    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a


In [5]:
def butter_bandpass_filter(data, lowcut=LOWCUT, highcut=HIGHCUT, fs=SAMPLE_RATE, order=5):
    """
    Filters the data using a Butterworth bandpass filter.
    
    Parameters
    ----------
    data : np.array
        The data to be filtered
        
    lowcut : int
        The lower frequency cutoff
    
    highcut : int
        The higher frequency cutoff
    
    fs : int
        The sample rate
    
    order : int
        The order of the filter
    """

    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [6]:
def butter_lowpass_filter(data, cutoff=LOWPASS_CUTOFF, fs = SAMPLE_RATE, order = 5):
    """
    Filters the data using a Butterworth bandpass filter.
    
    Parameters
    ----------
    data : np.array
        The data to be filtered
        
    cutoff : int
        The cutoff frequency of the filter
    
    fs : int
        The sample rate
    
    order : int
        The order of the filter
    """

    nyquist = 0.5 * fs
    cut = cutoff / nyquist
    b, a = butter(order, cut, btype='low')
    y = lfilter(b,a,data)
    return y

In [7]:
# Filter and detect peaks for each channel
def detect_wave_packet(channel_data):
    """
    Detects the peaks in a channel's data and returns the peak indices and the filtered signal.

    Parameters
    ----------
    channel_data : np.array
        The channel's data

    Returns
    -------
    peaks : np.array
        The peak indices

    filtered_signal : np.array
        The filtered signal
    """

    # Filter the channel data
    filtered_signal = butter_bandpass_filter(channel_data)

    # Find peaks in the filtered signal
    peaks, _ = find_peaks(filtered_signal, height=0.01, distance=100)

    return peaks, filtered_signal

## Experimentation

In [8]:
DATA = np.genfromtxt('sample1.csv', delimiter=',', skip_header=1)
RECTIFIED = np.absolute(DATA[:,1])
CHANNELS = [np.absolute(DATA[:,i+1]) for i in range(3)]
TIMES = DATA[:,0]

In [1]:
def get_spike_starts(rectified_data):
    filtered = butter_bandpass_filter(rectified_data, 50000, 58000)
    envelope = np.abs(hilbert(filtered))
    all_peaks = envelope[find_peaks(envelope)[0]]
    print(len(all_peaks))
    mean = np.mean(all_peaks)
    stdev = np.std(all_peaks)
    thresh = mean + 5*stdev
    thresh_peaks = []
    sample = 0
    while sample < len(filtered):
        if envelope[sample] > thresh:
            while envelope[sample-1] < envelope[sample]:
                sample -= 1
            thresh_peaks.append(sample)
            sample += SAMPLE_RATE
        sample += 1
    return filtered, np.array(thresh_peaks)

In [9]:
def get_filtered_spike_starts(data, sample_rate):
    all_peaks = data[find_peaks(data)[0]]
    mean = np.mean(all_peaks)
    stdev = np.std(all_peaks)
    thresh = mean + 5*stdev
    thresh_peaks = []
    sample = 0
    while sample < len(data):
        if data[sample] > thresh:
            thresh_peaks.append(sample)
            sample += sample_rate
        else:
            sample += 1
    return np.array(thresh_peaks)

def spectrogram_spikes(data, sample_rate):
    # filter
    filtered = butter_bandpass_filter(data, 50000, 58000)
    # get spectrogram
    spect = spectrogram(filtered, fs=sample_rate, nperseg=64)
    # use 58593.75khz freq
    signal1 = spect[2][6]
    sample_rate2 = spect[1].searchsorted(1)

    peaks = get_filtered_spike_starts(signal1, sample_rate2) * (sample_rate/sample_rate2)
    return peaks.astype(int), filtered


In [10]:
def scuffed_angle(data, sample_rate):
    spikes_1, _ = spectrogram_spikes(data[:,1], sample_rate)
    spikes_2, _ = spectrogram_spikes(data[:,2], sample_rate)
    mic_distance = 0.6 # meters
    max_t = 7.4*SAMPLE_RATE*mic_distance/SOUND_SPEED
    tds = np.array([spikes_2[i] - spikes_1[i] for i in range(len(spikes_1))]).clip(0, max_t)/max_t
    thetas = np.arccos(tds)
    print(max_t, tds * max_t)
    print(thetas)
    print(np.average(thetas)*180/math.pi)

In [12]:
get_spike_starts(DATA[:,1])

170492


(array([ 4.16532634e-09,  3.88375442e-08,  1.71568939e-07, ...,
        -1.99588178e-03, -1.33387093e-03, -3.04679445e-04]),
 array([ 1623305,  4183386,  6743416,  9303733, 11863425, 14423515]))

2024-01-26 18:35:08.787 Python[85485:7292811] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [15]:
scuffed_angle(DATA, SAMPLE_RATE)

1850.0 [672. 504. 504. 392. 616. 224.]
[1.19904977 1.29487614 1.29487614 1.35728592 1.23134161 1.44941742]
74.74088339578242


In [28]:
data = np.genfromtxt('sample1.csv', delimiter=',', skip_header=1)

In [16]:
scuffed_angle(data, SAMPLE_RATE)

1850.0 [ 840. 1850.  896.  840.  896. 1456.]
[1.0994861  0.         1.06520562 1.0994861  1.06520562 0.66482132]
47.6911425316116


In [13]:
filtered, peaks = get_spike_starts(RECTIFIED)

In [14]:
plt.plot(DATA[:,0], filtered)
plt.plot(DATA[peaks,0], np.zeros(len(peaks)), 'x')

2024-01-25 19:50:50.297 Python[70923:7138412] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [28]:
plt.plot(DATA[:,0], filtered)
plt.plot(DATA[thresh_peaks,0], np.zeros(len(thresh_peaks)), 'x')

## Neural Network

In [17]:
def load_peaks(filename):
    data = np.genfromtxt(filename, delimiter=',', skip_header=1)
    filtered1, peaks1 = get_spike_starts(data[:,1])
    filtered2, peaks2 = get_spike_starts(data[:,2])
    filtered3, peaks3 = get_spike_starts(data[:,3])
    times = data[:,0]
    return times, (peaks1, peaks2, peaks3), (filtered1, filtered2, filtered3)

In [23]:
loaded_model = keras.models.load_model('../models/tdoa_model.h5')  # Replace with the actual model path

In [ ]:
data = np.genfromtxt('sample6.csv', delimiter=',', skip_header=1)

In [19]:
filtered1, peaks1 = get_spike_starts(data[:,1])
filtered2, peaks2 = get_spike_starts(data[:,2])
filtered3, peaks3 = get_spike_starts(data[:,3])
times = data[:,0]

In [20]:
t1 = 0.0150591426641
t2 = 0.0150148075061
t3 = 0.0143952152545
expected_peaks2 = times.searchsorted(times[peaks1] + (t2-t1))
expected_peaks3 = times.searchsorted(times[peaks1] + (t3-t1))

In [27]:
# plot TIMES with filtered1, filtered2, filtered3 with their peaks on subplots
fig, axs = plt.subplots(3, 1)
axs[0].plot(times, filtered1)
axs[0].plot(times[peaks1], filtered1[peaks1], 'x')

axs[1].plot(times, filtered2)
axs[1].plot(times[peaks2], np.zeros(len(peaks2)), 'x')
axs[1].plot(times[expected_peaks2], np.zeros(len(expected_peaks2)), 'x')

axs[2].plot(times, filtered3)
axs[2].plot(times[peaks3], np.zeros(len(peaks3)), 'x')
axs[2].plot(times[expected_peaks3], np.zeros(len(expected_peaks3)), 'x')

In [24]:
difs = [
    [
        times[peaks1[i]] - times[peaks2[i]],
        times[peaks1[i]] - times[peaks3[i]]
    ] 
    for i in range(len(peaks1))
]
# difs = [
#     [0.00617665857168 - 0.00605309838017,
#     0.00617665857168 - 0.00553132493672]
# ]
predicted_thetas = loaded_model.predict(np.array(difs))
print(predicted_thetas * 180/math.pi)

1/1 [==============================] - 0s 52ms/step
[[ -77.90731]
 [-133.49117]
 [-131.43312]
 [ -96.65866]
 [-360.4488 ]
 [-354.90094]]


In [8]:
data = DATA

In [12]:
# filter
filtered1 = butter_bandpass_filter(data[:,1], 50000, 58000)
filtered2 = butter_bandpass_filter(data[:,2], 50000, 58000)
filtered3 = butter_bandpass_filter(data[:,3], 50000, 58000)
# get spectrogram
spect1 = spectrogram(filtered1, fs=SAMPLE_RATE, nperseg=64)
spect2 = spectrogram(filtered2, fs=SAMPLE_RATE, nperseg=64)
spect3 = spectrogram(filtered3, fs=SAMPLE_RATE, nperseg=64)
# use 58593.75khz freq
signal1 = spect1[2][6]
signal2 = spect2[2][6]
signal3 = spect3[2][6]

sample_rate = spect1[1].searchsorted(1)

peaks1 = get_filtered_spike_starts(signal1, sample_rate)
peaks2 = get_filtered_spike_starts(signal2, sample_rate)
peaks3 = get_filtered_spike_starts(signal3, sample_rate)


In [14]:
times = spect1[1]

In [15]:
difs = [
    [
        times[peaks1[i]] - times[peaks2[i]],
        times[peaks1[i]] - times[peaks3[i]]
    ] 
    for i in range(len(peaks1))
]
predicted_thetas = (loaded_model.predict(np.array(difs)) * 180/math.pi) % 360
print((predicted_thetas))
print(np.mean(predicted_thetas))

NameError: name 'loaded_model' is not defined

In [203]:
peaks1

array([ 27508,  73213, 118929, 164647, 210362, 256072])

In [25]:
plt.plot(spect1[1], signal1)
plt.plot(spect1[1][peaks1], np.zeros(len(peaks1)), 'x')

NameError: name 'spect1' is not defined

In [194]:
spect[0]

array([     0.   ,   9765.625,  19531.25 ,  29296.875,  39062.5  ,
        48828.125,  58593.75 ,  68359.375,  78125.   ,  87890.625,
        97656.25 , 107421.875, 117187.5  , 126953.125, 136718.75 ,
       146484.375, 156250.   , 166015.625, 175781.25 , 185546.875,
       195312.5  , 205078.125, 214843.75 , 224609.375, 234375.   ,
       244140.625, 253906.25 , 263671.875, 273437.5  , 283203.125,
       292968.75 , 302734.375, 312500.   ])

In [195]:
plt.plot(spect[2][6])

In [188]:
min_val = 5e-9
# get indices where spectrogram is above min_val
indices = np.unique(np.where(spect[2] > min_val)[0])
# print(spect[0][indices])
spect0 = spect[0][indices]
spect1 = spect[1]
spect2 = spect[2][indices]

In [191]:
plt.plot(spect2[0])

In [189]:
# plot spectrogram
plt.pcolormesh(spect1, spect0, spect2)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [139]:
# plot fft
plt.figure()
plt.plot(np.fft.fftfreq(len(fft), 1/SAMPLE_RATE), np.abs(fft))